# เซสชันที่ 5 – การจัดการหลายเอเจนต์

แสดงตัวอย่างการทำงานของสายงานแบบสองเอเจนต์อย่างง่าย (นักวิจัย -> บรรณาธิการ) โดยใช้ Foundry Local


### คำอธิบาย: การติดตั้ง Dependency
ติดตั้ง `foundry-local-sdk` และ `openai` ซึ่งจำเป็นสำหรับการเข้าถึงโมเดลในเครื่องและการสร้างข้อความแชท ผลลัพธ์จะเหมือนเดิมทุกครั้งที่ดำเนินการ


# สถานการณ์
การสร้างรูปแบบการทำงานแบบผู้ประสานงานสองตัวแทนที่เรียบง่าย:
- **ตัวแทนนักวิจัย** รวบรวมข้อมูลข้อเท็จจริงที่กระชับ
- **ตัวแทนบรรณาธิการ** เขียนใหม่ให้ชัดเจนสำหรับผู้บริหาร

แสดงให้เห็นถึงการใช้หน่วยความจำร่วมกันในแต่ละตัวแทน การส่งผ่านผลลัพธ์ระหว่างกลางแบบลำดับ และฟังก์ชันการทำงานแบบไปป์ไลน์ที่เรียบง่าย สามารถขยายไปยังบทบาทเพิ่มเติม (เช่น นักวิจารณ์, ผู้ตรวจสอบ) หรือสาขาคู่ขนานได้

**ตัวแปรสภาพแวดล้อม:**
- `FOUNDRY_LOCAL_ALIAS` - โมเดลเริ่มต้นที่จะใช้ (ค่าเริ่มต้น: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - โมเดลตัวแทนหลัก (แทนที่ ALIAS)
- `AGENT_MODEL_EDITOR` - โมเดลตัวแทนบรรณาธิการ (ค่าเริ่มต้นคือโมเดลหลัก)

**เอกสารอ้างอิง SDK:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**วิธีการทำงาน:**
1. **FoundryLocalManager** เริ่มต้นบริการ Foundry Local โดยอัตโนมัติ
2. ดาวน์โหลดและโหลดโมเดลที่ระบุ (หรือใช้เวอร์ชันที่แคชไว้)
3. ให้จุดเชื่อมต่อที่เข้ากันได้กับ OpenAI สำหรับการโต้ตอบ
4. ตัวแทนแต่ละตัวสามารถใช้โมเดลที่แตกต่างกันสำหรับงานเฉพาะทาง
5. มีตรรกะการลองใหม่ในตัวเพื่อจัดการกับความล้มเหลวชั่วคราวอย่างราบรื่น

**คุณสมบัติเด่น:**
- ✅ การค้นหาและเริ่มต้นบริการอัตโนมัติ
- ✅ การจัดการวงจรชีวิตของโมเดล (ดาวน์โหลด, แคช, โหลด)
- ✅ ความเข้ากันได้กับ OpenAI SDK สำหรับ API ที่คุ้นเคย
- ✅ รองรับหลายโมเดลสำหรับการทำงานเฉพาะทางของตัวแทน
- ✅ การจัดการข้อผิดพลาดที่แข็งแกร่งด้วยตรรกะการลองใหม่
- ✅ การประมวลผลในเครื่อง (ไม่ต้องใช้ API บนคลาวด์)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### คำอธิบาย: การนำเข้า Core & การใช้ Typing
แนะนำ dataclasses สำหรับการจัดเก็บข้อความของตัวแทนและการใช้ typing hints เพื่อความชัดเจน นำเข้า Foundry Local manager และ OpenAI client สำหรับการดำเนินการของตัวแทนในขั้นตอนถัดไป


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### คำอธิบาย: การเริ่มต้นใช้งานโมเดล (รูปแบบ SDK)
ใช้ Foundry Local Python SDK เพื่อการจัดการโมเดลที่มีประสิทธิภาพ:
- **FoundryLocalManager(alias)** - เริ่มต้นบริการอัตโนมัติและโหลดโมเดลตาม alias
- **get_model_info(alias)** - แปลง alias เป็น ID โมเดลที่เฉพาะเจาะจง
- **manager.endpoint** - ให้บริการ endpoint สำหรับ OpenAI client
- **manager.api_key** - ให้ API key (ไม่จำเป็นสำหรับการใช้งานในเครื่อง)
- รองรับโมเดลแยกสำหรับตัวแทนต่าง ๆ (หลัก vs ตัวแก้ไข)
- มีตรรกะการลองใหม่ในตัวพร้อมการเพิ่มเวลารอแบบทวีคูณเพื่อความยืดหยุ่น
- ตรวจสอบการเชื่อมต่อเพื่อให้แน่ใจว่าบริการพร้อมใช้งาน

**รูปแบบ SDK ที่สำคัญ:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**การจัดการวงจรชีวิต:**
- ผู้จัดการจะถูกเก็บไว้ในระดับโลกเพื่อการทำความสะอาดที่เหมาะสม
- ตัวแทนแต่ละตัวสามารถใช้โมเดลที่แตกต่างกันเพื่อความเชี่ยวชาญเฉพาะด้าน
- การค้นหาบริการและการจัดการการเชื่อมต่ออัตโนมัติ
- ลองใหม่อย่างราบรื่นพร้อมการเพิ่มเวลารอแบบทวีคูณเมื่อเกิดข้อผิดพลาด

สิ่งนี้ช่วยให้มั่นใจว่าการเริ่มต้นใช้งานถูกต้องก่อนที่การจัดการตัวแทนจะเริ่มต้นขึ้น

**อ้างอิง:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### คำอธิบาย: คลาส Agent & Memory
กำหนด `AgentMsg` ที่มีน้ำหนักเบาสำหรับรายการหน่วยความจำ และ `Agent` ที่ครอบคลุม:
- **บทบาทของระบบ** - บุคลิกและคำแนะนำของ Agent
- **ประวัติข้อความ** - รักษาบริบทของการสนทนา
- **เมธอด act()** - ดำเนินการด้วยการจัดการข้อผิดพลาดอย่างเหมาะสม

Agent สามารถใช้โมเดลที่แตกต่างกัน (หลัก vs ตัวแก้ไข) และรักษาบริบทแยกกันสำหรับแต่ละ Agent รูปแบบนี้ช่วยให้:
- การคงหน่วยความจำระหว่างการดำเนินการ
- การกำหนดโมเดลที่ยืดหยุ่นสำหรับแต่ละ Agent
- การแยกข้อผิดพลาดและการกู้คืน
- การเชื่อมโยงและการจัดการที่ง่ายดาย


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### คำอธิบาย: กระบวนการแบบมีการจัดการ
สร้างตัวแทนเฉพาะทางสองตัว:
- **นักวิจัย**: ใช้โมเดลหลัก รวบรวมข้อมูลข้อเท็จจริง
- **บรรณาธิการ**: สามารถใช้โมเดลแยกต่างหาก (ถ้ากำหนดค่าไว้) ปรับปรุงและเขียนใหม่

ฟังก์ชัน `pipeline`:
1. นักวิจัยรวบรวมข้อมูลดิบ
2. บรรณาธิการปรับปรุงข้อมูลให้พร้อมสำหรับการใช้งานระดับผู้บริหาร
3. ส่งคืนผลลัพธ์ทั้งแบบขั้นกลางและขั้นสุดท้าย

รูปแบบนี้ช่วยให้:
- การปรับแต่งโมเดล (ใช้โมเดลต่างกันสำหรับบทบาทต่างๆ)
- การปรับปรุงคุณภาพผ่านการประมวลผลหลายขั้นตอน
- การตรวจสอบย้อนกลับของการเปลี่ยนแปลงข้อมูล
- การขยายตัวที่ง่ายขึ้นสำหรับตัวแทนเพิ่มเติมหรือการประมวลผลแบบขนาน


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### คำอธิบาย: การดำเนินการและผลลัพธ์ของ Pipeline
ดำเนินการ pipeline แบบหลายตัวแทนบนคำถามที่เกี่ยวกับการปฏิบัติตามกฎระเบียบและความหน่วงเวลา เพื่อแสดงให้เห็นถึง:
- การเปลี่ยนแปลงข้อมูลหลายขั้นตอน
- ความเชี่ยวชาญและการทำงานร่วมกันของตัวแทน
- การปรับปรุงคุณภาพผลลัพธ์ผ่านการกลั่นกรอง
- การตรวจสอบย้อนกลับ (ทั้งผลลัพธ์ระหว่างทางและผลลัพธ์สุดท้ายถูกเก็บรักษาไว้)

**โครงสร้างผลลัพธ์:**
- `question` - คำถามต้นฉบับจากผู้ใช้
- `research` - ผลลัพธ์การวิจัยดิบ (ข้อมูลข้อเท็จจริงแบบหัวข้อย่อย)
- `final` - สรุปผลที่ผ่านการกลั่นกรอง
- `models` - โมเดลที่ใช้ในแต่ละขั้นตอน

**แนวคิดการขยายเพิ่มเติม:**
1. เพิ่มตัวแทน Critic สำหรับการตรวจสอบคุณภาพ
2. ใช้ตัวแทนวิจัยแบบขนานสำหรับแง่มุมต่าง ๆ
3. เพิ่มตัวแทน Verifier สำหรับการตรวจสอบข้อเท็จจริง
4. ใช้โมเดลที่แตกต่างกันสำหรับระดับความซับซ้อนที่ต่างกัน
5. ใช้กระบวนการป้อนกลับเพื่อปรับปรุงแบบวนซ้ำ


### ขั้นสูง: การตั้งค่าตัวแทนแบบกำหนดเอง

ลองปรับแต่งพฤติกรรมของตัวแทนโดยการแก้ไขตัวแปรสภาพแวดล้อมก่อนที่จะรันเซลล์การเริ่มต้น:

**โมเดลที่มีให้ใช้งาน:**
- ใช้ `foundry model ls` ในเทอร์มินัลเพื่อดูโมเดลทั้งหมดที่มีให้ใช้งาน
- ตัวอย่าง: phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b เป็นต้น


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้อง แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษามนุษย์ที่มีความเชี่ยวชาญ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดที่เกิดจากการใช้การแปลนี้
